# Import Important Libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import splitfolders
from tensorflow import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from keras.layers import *
from keras.preprocessing import *
from keras.preprocessing.image import *
from tensorflow import keras
import pandas as pd
import numpy as np
import seaborn as sns
import random as random
import matplotlib as mpl
import warnings
import os
import cv2
from pathlib import Path

# Ignore warnings

In [ ]:
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 


# Set Random Values

In [ ]:
seed_value=40
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Define Some Parameters 

In [ ]:
Image_Height = 128  
Image_Width = 128
Batch_Size = 64
ImagePath = './Kideny/'

# Count Number Of Images

In [ ]:
Data_Root = Path(ImagePath)

Total = 0
for sub_dir in Data_Root.iterdir():
    Count = len(list(sub_dir.iterdir()))
    Total += Count
    print(f'{sub_dir.name}: {Count}')

print('')
print(f'Total Numbers Of Files: {Total}')

# Split The Data Into Train, Test, And Validation

In [ ]:
os.mkdir('Data')

In [ ]:
Input = ImagePath
Output = './Data'

splitfolders.ratio(Input, output=Output, seed=seed_value, ratio=(.80, 0.1, .1))

# Generate Image DataSet

In [ ]:
Training_Directory = './Data/train'

Testing_Directory = './Data/test'

Validation_Directory = './Data/val'

In [ ]:
Training_Set = tf.keras.preprocessing.image_dataset_from_directory(directory=Training_Directory,
                                                                   image_size=(Image_Height, Image_Width),
                                                                   batch_size=Batch_Size,
                                                                   label_mode='categorical',
                                                                   shuffle=True,
                                                                   seed= seed_value,
                                                                   color_mode='grayscale',
                                                                   labels='inferred',
                                                                   class_names = ['Cyst', 'Normal', 'Stone', 'Tumor']
                                                                   )

In [ ]:
Testing_Set = tf.keras.preprocessing.image_dataset_from_directory(directory=Testing_Directory,
                                                                   image_size=(Image_Height, Image_Width),
                                                                   batch_size=Batch_Size,
                                                                   label_mode='categorical',
                                                                   shuffle=True,
                                                                   seed= seed_value,
                                                                   color_mode='grayscale',
                                                                   labels='inferred',
                                                                   class_names = ['Cyst', 'Normal', 'Stone', 'Tumor']
                                                                   )

In [ ]:
Validation_Set = tf.keras.preprocessing.image_dataset_from_directory(directory=Validation_Directory,
                                                                   image_size=(Image_Height, Image_Width),
                                                                   batch_size=Batch_Size,
                                                                   label_mode='categorical',
                                                                   shuffle=True,
                                                                   seed= seed_value,
                                                                   color_mode='grayscale',
                                                                   labels='inferred',
                                                                   class_names = ['Cyst', 'Normal', 'Stone', 'Tumor']
                                                                   )

# Data Visualization 

In [ ]:
def plot_some_images(Data, NumberOfImages):
    
    '''Plot Some Images From Training Set, Testing Set And Validation Set With Class Of Selected Image'''

    Labels = Data.class_names
    Data = Data
    NumberOfImages = NumberOfImages

    Row = Col  = int(tf.get_static_value(math.ceil(NumberOfImages**0.5)))

    Counter = 1

    plt.figure(figsize=(15, 15))
    for images, labels in Data:
     for i in range(NumberOfImages):
        ax = plt.subplot(Row, Col,Counter)
        plt.imshow(images[i], cmap='gray')
        plt.title(Labels[np.argmax(labels[i])], color='green', fontsize = 15)
        plt.axis("off")

        if Counter == NumberOfImages:
           break
        else:
           Counter += 1

**1. Training Set**

In [ ]:
plot_some_images(Training_Set, 20)

**2.Testing Set**

In [ ]:
plot_some_images(Testing_Set, 20)

**3. Validation Set**

In [ ]:
plot_some_images(Validation_Set, 20)

# Models

In [ ]:
Input_Shape = (Image_Height, Image_Width, 1)
Epochs = 100
Number_Of_Classes = len(os.listdir(ImagePath))

In [ ]:
normalization_layer = Rescaling(1./255)

In [ ]:
Normalized_Training_Set = Training_Set.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
Callback = tf.keras.callbacks.EarlyStopping(
                    monitor="accuracy",
                    patience=3,
                    verbose=0,
                    mode="max",
                    restore_best_weights=True,
                    start_from_epoch=0
                )

In [ ]:
Predictin_Image1 = r'./Prediction Images/Normal- (30).jpg'
Predictin_Image2 = r'./Prediction Images/Cyst- (22).jpg'
Predictin_Image3 = r'./Prediction Images/Stone- (5).jpg'
Predictin_Image4 = r'./Prediction Images/Tumor- (20).jpg'
Predictin_Image5 = r'./Prediction Images/Normal- (76).jpg'
Predictin_Image6 = r'./Prediction Images/Stone- (26).jpg'
Predictin_Image7 = r'./Prediction Images/Cyst- (38).jpg'
Predictin_Image8 = r'./Prediction Images/Tumor- (9).jpg'


In [ ]:
class Models(): 
    def __init__(self,input_shape,epochs) :
        self.input_shape = input_shape
        self.epochs = epochs
        
    def convolutional_neural_network(self):
        '''Build Convolutional Neural Network For Images Classification, Compile It, Anf Fit It'''
        model = keras.models.Sequential([
                                         Flatten(),
                                         Dense(128 ,activation='relu'),
                                         Dense(Number_Of_Classes, activation='softmax')])
            
        model.compile(loss= 'categorical_crossentropy',
                        optimizer='adam',
                        metrics=['accuracy'])

        
        model.fit(Normalized_Training_Set, 
                  epochs=self.epochs, 
                  validation_data=Validation_Set,
                  callbacks=[Callback],
                  steps_per_epoch=None,
                  shuffle=True,
                  validation_batch_size = None,
                  batch_size=None,
                  initial_epoch	 = 0)
        
        return model
    

    def tranfer_learning(self, base_model):  
        '''Define Transfer Learning Models For Images Classification, Compile It, Anf Fit It'''

        base_model = base_model
        base_model.trainable = False

        for layer in base_model.layers[:-4]:
            layer.trainable = True

        inputs = tf.keras.Input(self.input_shape)
        GrayScaleImage = tf.keras.layers.Concatenate()([inputs, inputs, inputs])
        x = base_model(GrayScaleImage)
        outputs = Dense(Number_Of_Classes, 'softmax')(x)
        model = tf.keras.Model(inputs, outputs)
          
        model.compile(loss= 'categorical_crossentropy',
                        optimizer='adam',
                        metrics=['accuracy'])

        model.fit(Normalized_Training_Set, 
                  epochs=self.epochs, 
                  validation_data=Validation_Set,
                  callbacks=[Callback],
                  steps_per_epoch=len(Training_Set),
                  shuffle=True,
                  validation_batch_size = len(Validation_Set),
                  batch_size=Batch_Size,
                  initial_epoch	= 0)
        
        return model
    

# Model History Result

In [ ]:
def model_history_values(model, name):
    '''Export The Model Training Accuracy, Training Loss, Validation Accuracy, Validation Loss To DataFrame And CSV File'''
    file_name = name
    HistoryResultDF= pd.DataFrame(model.history.history)
    HistoryResultDF.to_csv(os.path.join('{}HitoryResult.csv'.format(file_name)) ,index=False)
    return HistoryResultDF

# Ploting Model History

In [ ]:
def plot_model_history(ModelHistory): 
        '''Plot The Model Training Accuracy, Training Loss, Validation Accuracy, Validation Loss '''
        figure, ax = plt.subplots(2, 1, figsize=(10, 10))
        ModelHistory[['accuracy', 'val_accuracy']].plot(ax=ax[0],title='Accuracy');
        ModelHistory[['loss', 'val_loss']].plot(ax=ax[1], title='Loss');


# Find Model Values

In [ ]:
class ModelValues():
        def __init__(self, HistoryResultDF) :
                self.HistoryResultDF = HistoryResultDF

        def loss_accuracy_values(self):
                '''Find The Mean Value Of The Model Training Accuracy, Training Loss, Validation Accuracy, Validation Loss And Export As Dataframe'''

                Loss = self.HistoryResultDF['loss'].mean()
                Accuracy = self.HistoryResultDF['accuracy'].mean()
                ValLoss = self.HistoryResultDF['val_loss'].mean()
                ValAccuracy = self.HistoryResultDF['val_accuracy'].mean()
                LossAccuracyValuesDf =  pd.DataFrame(data=[Loss, Accuracy, ValLoss, ValAccuracy], index=['Loss', 'Accuracy', 'Val_Loss', 'Val_Accuracy'], columns=['Values'])

                return Loss, Accuracy, ValLoss, ValAccuracy, LossAccuracyValuesDf

        def loss_accuracy_valuesـmaximum_values(self):
                '''Find The Maximum Value Of The Model Training Accuracy, Training Loss, Validation Accuracy, Validation Loss And Export As Dataframe'''

                MaxLoss = self.HistoryResultDF['loss'].max()
                MaxAccuracy = self.HistoryResultDF['accuracy'].max()
                MaxValLoss = self.HistoryResultDF['val_loss'].max()
                MaxValAccuracy = self.HistoryResultDF['val_accuracy'].max()
                MaxLossAccuracyValuesDf =  pd.DataFrame(data=[MaxLoss, MaxAccuracy, MaxValLoss, MaxValAccuracy], index=['Max_Loss', 'Max_Accuracy', 'Max_Val_Loss', 'Max_Val_Accuracy'], columns=['Values'])

                return MaxLoss, MaxAccuracy, MaxValLoss, MaxValAccuracy, MaxLossAccuracyValuesDf

        def loss_accuracy_valuesـminimum_values(self):
                '''Find The Minimum Value Of The Model Training Accuracy, Training Loss, Validation Accuracy, Validation Loss And Export As Dataframe'''

                MinLoss = self.HistoryResultDF['loss'].min()
                MinAccuracy = self.HistoryResultDF['accuracy'].min()
                MinValLoss = self.HistoryResultDF['val_loss'].min()
                MinValAccuracy = self.HistoryResultDF['val_accuracy'].min()
                MinLossAccuracyValuesDf =  pd.DataFrame(data=[MinLoss, MinAccuracy, MinValLoss, MinValAccuracy], index=['Min_Loss', 'Min_Accuracy', 'Min_Val_Loss', 'Min_Val_Accuracy'], columns=['Values'])

                return MinLoss, MinAccuracy, MinValLoss, MinValAccuracy, MinLossAccuracyValuesDf

# Model Evaluation

In [ ]:
def model_evulation(model, data):
    '''Find The Testing Accuracy And Testing Loss And Export As Dataframe '''
    Loss, Accuracy = model.evaluate(data, verbose=0)

    EvaluationValues = pd.DataFrame(index= ['Loss', 'Accuracy'], data = [Loss, Accuracy], columns=['Values'])
    
    return Loss, Accuracy, EvaluationValues

# Generate Real And Prediction Classes

In [ ]:
def generate_real_prediction_classes(model):
    '''Generate The Real And Predection Classes For Use In Confusion Matrix And Performance Metrics '''

    Real =  np.array([])
    Predictions = np.array([])
    
    for x, y in Testing_Set:
        Real = np.concatenate((Real, np.argmax(y, axis=1)))
        y_pred = np.argmax(model.predict(x, verbose=0),axis=1)
        Predictions = np.concatenate([Predictions, y_pred])
        
        if len(Real) == len(Testing_Set.file_paths):
            break

    return Real, Predictions

# Confusion Matrix

In [ ]:
def confusion_matrix_(Real, Prediction):
        '''The confusion matrix is used to analyze the ability of model to identify the classes of the dataset'''

        ConfusionMatrix = confusion_matrix(Real, Prediction)
    
        return ConfusionMatrix
    
def plot_confusion_matrix(ConfusionMatrix, name):
    '''Display The Confusion Matrix'''
    
    disp = ConfusionMatrixDisplay(confusion_matrix=ConfusionMatrix, display_labels = Testing_Set.class_names)
    disp.plot(cmap='YlGnBu', colorbar=False, xticks_rotation='vertical', values_format='d')
    plt.title('{} Confusion Matrix with Labels'.format(name))
    plt.rcParams['font.size'] = '10'
    plt.grid(None)
    return plt.show();

# Model Performance Metrics

**1. Generate True Positive, False Positive, True Negative, False Negative**

*A. Positive*

In [ ]:
class Generate_Positive():     
    def __init__(self, Confusin_Matrix):
        self.Confusin_Matrix = Confusin_Matrix
    
    def false_positive(self):
        '''The prediction is wrong and the real value is positive 
        Using numpy to find the values of false positive from confusion matrix'''

        False_Postive = self.Confusin_Matrix.sum(axis=0) - np.diag(self.Confusin_Matrix)
        False_Postive = False_Postive.astype(int)
        False_Postive = sum(False_Postive)
        
        return False_Postive
        
    def true_positive(self):
        ''' The prediction is correct and the real value is positive 
        Using numpy to find the values of true positive from confusion matrix'''

        True_Positive = np.diag(self.Confusin_Matrix)
        True_Positive = True_Positive.astype(int)
        True_Positive = sum(True_Positive)
        
        return True_Positive


*B. Negative*

In [ ]:
class Generate_Negative():
    
    def __init__(self, Confusin_Matrix):
        self.Confusin_Matrix = Confusin_Matrix
    
    def false_negative(self):
        '''The prediction is wrong and the real value is negative 
        Using numpy to find the values of false negative from confusion matrix'''

        False_Negative = self.Confusin_Matrix.sum(axis=1) - np.diag(self.Confusin_Matrix)
        False_Negative = False_Negative.astype(int)
        False_Negative = sum(False_Negative)
        
        return False_Negative 
    
    def true_negative(self):
         ''' The prediction is correct and the real value is negative 
         Using numpy to find the values of true negative from confusion matrix'''

         True_Negative = self.Confusin_Matrix.sum()
         X = (self.Confusin_Matrix.sum(axis=0) - np.diag(self.Confusin_Matrix)) + (self.Confusin_Matrix.sum(axis=1) - np.diag(self.Confusin_Matrix)) + np.diag(self.Confusin_Matrix)
         True_Negative = True_Negative.astype(int) - X
         True_Negative = sum(True_Negative)
         
         return True_Negative

**2. Performance Merics**

In [ ]:
class Performance_Metrics:
    
    def __init__(self, True_Positive, True_Negative, False_Positive, False_Negative):
        self.True_Positive = True_Positive
        self.True_Negative = True_Negative
        self.False_Positive = False_Positive
        self.False_Negative = False_Negative
        
            
    def accuracy_score(self):
        ''' Accuracy is using to measure the accurate of the model '''
        Accuracy = (self.True_Positive + self.True_Negative) / (self.True_Positive + self.True_Negative + self.False_Positive + self.False_Negative)

        return Accuracy
    
    def precision_score(self):
        '''Precision is used to identified is the predicted result is close to the real result'''
        
        Precision = ((self.True_Positive) / (self.True_Positive + self.False_Positive))
        
        return Precision
    
    def sensitivity_score(self):
        '''Sensitivity is true positive rate measure'''

        Sensitivity = ((self.True_Positive ) / (self.True_Positive  + self.False_Negative)) 

        return Sensitivity
     
    def specificity_score(self):
        '''Specificity is true negative rate measure'''

        Specificity = ((self.True_Negative) / (self.True_Negative + self.False_Negative)) 

        return Specificity
    
    def f1_score(self):
        '''F-Measure refers to the mean of consistency between precision and sensitivity'''
        
        F1Score = (((self.True_Positive ) / (self.True_Positive  + self.False_Negative) * (self.True_Positive) / (self.True_Positive + self.False_Positive)) / ((self.True_Positive ) / (self.True_Positive  + self.False_Negative) + (self.True_Positive) / (self.True_Positive + self.False_Positive))) * 2

        return F1Score
    
    def err_rate(self):
        Error = ((self.False_Positive + self.False_Negative) / (self.True_Positive + self.True_Negative + self.False_Positive + self.False_Negative)) 
  
        return Error 
    
    @staticmethod 
    def display_performance_metrics_values(Accuracy, Precision, Sensitivity, Specificity, F1Score, Error):
        MetricsValues = pd.DataFrame(index=['Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'F1Score', 'Error'], columns=['Values'], data=[Accuracy, Precision, Sensitivity, Specificity, F1Score, Error])

        return MetricsValues   
    

# Predict Images

In [ ]:
def predict(prediction):
    if np.argmax(prediction) == 0:
        print('Cyst')
    elif np.argmax(prediction) == 1:
        print('Normal')   
    elif np.argmax(prediction) == 2:
        print('Stone')    
    elif np.argmax(prediction) == 3:
        print('Tumor')
    else:
        print('Unkown')

def predict_image(model, path):

    PredictionImage = tf.keras.utils.load_img(path, target_size=(Image_Height, Image_Width, 1))
    PredictionImageArray = tf.keras.utils.img_to_array(PredictionImage)
    PredictionImageArray = np.array([PredictionImageArray])
    GrayScaleImages = tf.image.rgb_to_grayscale(PredictionImageArray)
    predictions = model.predict(GrayScaleImages)
    Predction = predict(predictions)
    return Predction

# Convolutional Neural Network

In [ ]:
name = 'Convolutional Neural Network'

In [ ]:
with tf.device('GPU'):
    model = Models(Input_Shape, Epochs)
    model0 = model.convolutional_neural_network()

In [ ]:
HistoryResultDF0 = model_history_values(model0, name)

In [ ]:
HistoryResultDF0

In [ ]:
plot_model_history(HistoryResultDF0)

In [ ]:
ModelValues0 = ModelValues(HistoryResultDF0)
Loss0, Accuracy0, ValLoss0, ValAccuracy0, LossAccuracyValuesDf0 = ModelValues0.loss_accuracy_values()
MaxLoss0, MaxAccuracy0, MaxValLoss0, MaxValAccuracy0, MaxLossAccuracyValuesDf0 = ModelValues0.loss_accuracy_valuesـmaximum_values()
MinLoss0, MinAccuracy0, MinValLoss0, MinValAccuracy0, MinLossAccuracyValuesDf0 = ModelValues0.loss_accuracy_valuesـminimum_values()

In [ ]:
LossAccuracyValuesDf0

In [ ]:
MaxLossAccuracyValuesDf0

In [ ]:
MinLossAccuracyValuesDf0

In [ ]:
TestingLoss0, TestingAccuracy0, EvaluationValues0 = model_evulation(model0, Testing_Set)

In [ ]:
EvaluationValues0

In [ ]:
Real0, Predictions0 = generate_real_prediction_classes(model0)

In [ ]:
ConfusionMatrix0 = confusion_matrix_(Real0, Predictions0)

In [ ]:
plot_confusion_matrix(ConfusionMatrix0, name)

In [ ]:
Positive0 = Generate_Positive(ConfusionMatrix0)
TruePositive0 = Positive0.true_positive()
FalsePostive0 = Positive0.false_positive()

In [ ]:
Negative0 = Generate_Negative(ConfusionMatrix0)
TrueNegative0 = Negative0.true_negative()
FalseNegative0 = Negative0.false_negative()

In [ ]:
Performance0 = Performance_Metrics(TruePositive0, TrueNegative0 ,FalsePostive0, FalseNegative0)

Accuracy0 = Performance0.accuracy_score()
Precision0 = Performance0.precision_score()
Sensitivity0 = Performance0.sensitivity_score()
Specificity0 = Performance0.specificity_score()
F1Score0 = Performance0.f1_score()
ErrorRate0 = Performance0.err_rate()

PerformanceValues0 = Performance0.display_performance_metrics_values(Accuracy0, Precision0, Sensitivity0, Specificity0, F1Score0, ErrorRate0)

In [ ]:
PerformanceValues0

In [ ]:
predict_image(model0, Predictin_Image1)

In [ ]:
predict_image(model0, Predictin_Image2)

In [ ]:
predict_image(model0, Predictin_Image3)

In [ ]:
predict_image(model0, Predictin_Image4)

In [ ]:
predict_image(model0, Predictin_Image5)

In [ ]:
predict_image(model0, Predictin_Image6)

In [ ]:
predict_image(model0, Predictin_Image7)

In [ ]:
predict_image(model0, Predictin_Image8)

# Transfer Learning

**1. InceptionV3**

In [ ]:
name = 'InceptionV3'

In [ ]:
with tf.device('GPU'):
    base_model = tf.keras.applications.inception_v3.InceptionV3(
                    include_top=False,
                    weights='imagenet',
                    pooling = 'avg',
                    )
    
    Inc = Models(Input_Shape, Epochs)
    model1 = Inc.tranfer_learning(base_model)

In [ ]:
HistoryResultDF1 = model_history_values(model1, name)

In [ ]:
HistoryResultDF1

In [ ]:
plot_model_history(HistoryResultDF1)

In [ ]:
ModelValues1 = ModelValues(HistoryResultDF1)
Loss1, Accuracy1, ValLoss1, ValAccuracy1, LossAccuracyValuesDf1 = ModelValues1.loss_accuracy_values()
MaxLoss1, MaxAccuracy1, MaxValLoss1, MaxValAccuracy1, MaxLossAccuracyValuesDf1 = ModelValues1.loss_accuracy_valuesـmaximum_values()
MinLoss1, MinAccuracy1, MinValLoss1, MinValAccuracy1, MinLossAccuracyValuesDf1 = ModelValues1.loss_accuracy_valuesـminimum_values()

In [ ]:
LossAccuracyValuesDf1

In [ ]:
MaxLossAccuracyValuesDf1

In [ ]:
MinLossAccuracyValuesDf1

In [ ]:
TestingLoss1, TestingAccuracy1, EvaluationValues1 = model_evulation(model1, Testing_Set)

In [ ]:
EvaluationValues1

In [ ]:
Real1, Predictions1 = generate_real_prediction_classes(model1)

In [ ]:
ConfusionMatrix1 = confusion_matrix_(Real1, Predictions1)

In [ ]:
Positive1 = Generate_Positive(ConfusionMatrix1)
TruePositive1 = Positive1.true_positive()
FalsePostive1 = Positive1.false_positive()

In [ ]:
Negative1 = Generate_Negative(ConfusionMatrix1)
TrueNegative1 = Negative1.true_negative()
FalseNegative1 = Negative1.false_negative()

In [ ]:
Performance1 = Performance_Metrics(TruePositive1, TrueNegative1 ,FalsePostive1, FalseNegative1)

Accuracy1 = Performance1.accuracy_score()
Precision1 = Performance1.precision_score()
Sensitivity1 = Performance1.sensitivity_score()
Specificity1 = Performance1.specificity_score()
F1Score1 = Performance1.f1_score()
ErrorRate1 = Performance1.err_rate()

PerformanceValues1 = Performance1.display_performance_metrics_values(Accuracy1, Precision1, Sensitivity1, Specificity1, F1Score1, ErrorRate1)

In [ ]:
PerformanceValues1

In [ ]:
predict_image(model1, Predictin_Image1)

In [ ]:
predict_image(model1, Predictin_Image2)

In [ ]:
predict_image(model1, Predictin_Image3)

In [ ]:
predict_image(model1, Predictin_Image4)

In [ ]:
predict_image(model1, Predictin_Image5)

In [ ]:
predict_image(model1, Predictin_Image6)

In [ ]:
predict_image(model1, Predictin_Image7)

In [ ]:
predict_image(model1, Predictin_Image8)

**2. DenseNet121**

In [ ]:
name = 'DenseNet121'

In [84]:
with tf.device('GPU'):
        base_model = tf.keras.applications.densenet.DenseNet121(
                include_top=False,
                weights='imagenet',
                 pooling = 'avg',                    
                )

        NetV2B0 = Models(Input_Shape, Epochs)
        model2 = NetV2B0.tranfer_learning(base_model)

In [ ]:
HistoryResultDF2 = model_history_values(model2, name)

In [ ]:
HistoryResultDF2

In [ ]:
plot_model_history(HistoryResultDF2)

In [ ]:
ModelValues2 = ModelValues(HistoryResultDF2)
Loss2, Accuracy2, ValLoss2, ValAccuracy2, LossAccuracyValuesDf2 = ModelValues2.loss_accuracy_values()
MaxLoss2, MaxAccuracy2, MaxValLoss2, MaxValAccuracy2, MaxLossAccuracyValuesDf2 = ModelValues2.loss_accuracy_valuesـmaximum_values()
MinLoss2, MinAccuracy2, MinValLoss2, MinValAccuracy2, MinLossAccuracyValuesDf2 = ModelValues2.loss_accuracy_valuesـminimum_values()

In [ ]:
LossAccuracyValuesDf2

In [ ]:
MaxLossAccuracyValuesDf2

In [ ]:
MinLossAccuracyValuesDf2

In [ ]:
TestingLoss2, TestingAccuracy2, EvaluationValues2 = model_evulation(model2, Testing_Set)

In [ ]:
EvaluationValues2

In [ ]:
Real2, Predictions2 = generate_real_prediction_classes(model2)

In [ ]:
ConfusionMatrix2 = confusion_matrix_(Real2, Predictions2)

In [ ]:
plot_confusion_matrix(ConfusionMatrix2, name)

In [ ]:
Positive2 = Generate_Positive(ConfusionMatrix2)
TruePositive2= Positive2.true_positive()
FalsePostive2 = Positive2.false_positive()

In [ ]:
Negative2 = Generate_Negative(ConfusionMatrix2)
TrueNegative2 = Negative2.true_negative()
FalseNegative2 = Negative2.false_negative()

In [ ]:
Performance2 = Performance_Metrics(TruePositive2, TrueNegative2 ,FalsePostive2, FalseNegative2)

Accuracy2 = Performance2.accuracy_score()
Precision2 = Performance2.precision_score()
Sensitivity2 = Performance2.sensitivity_score()
Specificity2 = Performance2.specificity_score()
F1Score2 = Performance2.f1_score()
ErrorRate2 = Performance2.err_rate()

PerformanceValues2 = Performance2.display_performance_metrics_values(Accuracy2, Precision2, Sensitivity2, Specificity2, F1Score2, ErrorRate2)

In [ ]:
PerformanceValues2

In [ ]:
predict_image(model2, Predictin_Image1)

In [ ]:
predict_image(model2, Predictin_Image2)

In [ ]:
predict_image(model2, Predictin_Image3)

In [ ]:
predict_image(model2, Predictin_Image4)

In [ ]:
predict_image(model2, Predictin_Image5)

In [ ]:
predict_image(model2, Predictin_Image6)

In [ ]:
predict_image(model2, Predictin_Image7)

In [ ]:
predict_image(model2, Predictin_Image8)

**3. MobileNet**

In [ ]:
name = 'MobileNet'

In [ ]:
with tf.device('GPU'):
        base_model = tf.keras.applications.mobilenet.MobileNet(
                include_top=False,
                weights='imagenet',
                pooling = 'avg',
                )

        Mobile = Models(Input_Shape, Epochs)
        model3 = Mobile.tranfer_learning(base_model)

In [ ]:
HistoryResultDF3 = model_history_values(model3, name)

In [ ]:
HistoryResultDF3

In [ ]:
plot_model_history(HistoryResultDF3)

In [ ]:
ModelValues3 = ModelValues(HistoryResultDF3)
Loss3, Accuracy3, ValLoss3, ValAccuracy3, LossAccuracyValuesDf3 = ModelValues3.loss_accuracy_values()
MaxLoss3, MaxAccuracy3, MaxValLoss3, MaxValAccuracy3, MaxLossAccuracyValuesDf3 = ModelValues3.loss_accuracy_valuesـmaximum_values()
MinLoss3, MinAccuracy3, MinValLoss3, MinValAccuracy3, MinLossAccuracyValuesDf3 = ModelValues3.loss_accuracy_valuesـminimum_values()

In [ ]:
LossAccuracyValuesDf3

In [ ]:
MaxLossAccuracyValuesDf3

In [ ]:
MinLossAccuracyValuesDf3

In [ ]:
TestingLoss3, TestingAccuracy3, EvaluationValues3 = model_evulation(model3, Testing_Set)

In [ ]:
EvaluationValues3

In [ ]:
Real3, Predictions3 = generate_real_prediction_classes(model3)

In [ ]:
ConfusionMatrix3 = confusion_matrix_(Real3, Predictions3)

In [ ]:
plot_confusion_matrix(ConfusionMatrix3, name)

In [ ]:
Positive3 = Generate_Positive(ConfusionMatrix3)
TruePositive3 = Positive3.true_positive()
FalsePostive3 = Positive3.false_positive()

In [ ]:
Negative3 = Generate_Negative(ConfusionMatrix3)
TrueNegative3 = Negative3.true_negative()
FalseNegative3 = Negative3.false_negative()

In [ ]:
Performance3 = Performance_Metrics(TruePositive3, TrueNegative3 ,FalsePostive3, FalseNegative3)

Accuracy3 = Performance3.accuracy_score()
Precision3 = Performance3.precision_score()
Sensitivity3 = Performance3.sensitivity_score()
Specificity3 = Performance3.specificity_score()
F1Score3 = Performance3.f1_score()
ErrorRate3 = Performance3.err_rate()

PerformanceValues3 = Performance3.display_performance_metrics_values(Accuracy3, Precision3, Sensitivity3, Specificity3, F1Score3, ErrorRate3)

In [ ]:
PerformanceValues3

In [ ]:
predict_image(model3, Predictin_Image1)

In [ ]:
predict_image(model3, Predictin_Image2)

In [ ]:
predict_image(model3, Predictin_Image3)

In [ ]:
predict_image(model3, Predictin_Image4)

In [ ]:
predict_image(model3, Predictin_Image5)

In [ ]:
predict_image(model3, Predictin_Image6)

In [ ]:
predict_image(model3, Predictin_Image7)

In [ ]:
predict_image(model3, Predictin_Image8)

**4. Xception**

In [ ]:
name = 'Xception'

In [ ]:
with tf.device('GPU'):
        base_model = tf.keras.applications.xception.Xception(
                include_top=False,
                weights='imagenet',
                pooling = 'avg',
        )

        XCE = Models(Input_Shape, Epochs)
        model4 = XCE.tranfer_learning(base_model)

In [ ]:
HistoryResultDF4= model_history_values(model4, name)

In [ ]:
HistoryResultDF4

In [ ]:
plot_model_history(HistoryResultDF4)

In [ ]:
ModelValues4 = ModelValues(HistoryResultDF4)
Loss4, Accuracy4, ValLoss4, ValAccuracy4, LossAccuracyValuesDf4 = ModelValues4.loss_accuracy_values()
MaxLoss4, MaxAccuracy4, MaxValLoss4, MaxValAccuracy4, MaxLossAccuracyValuesDf4 = ModelValues4.loss_accuracy_valuesـmaximum_values()
MinLoss4, MinAccuracy4, MinValLoss4, MinValAccuracy4, MinLossAccuracyValuesDf4 = ModelValues4.loss_accuracy_valuesـminimum_values()

In [ ]:
LossAccuracyValuesDf4

In [ ]:
MaxLossAccuracyValuesDf4

In [ ]:
MinLossAccuracyValuesDf4

In [ ]:
TestingLoss4, TestingAccuracy4, EvaluationValues4 = model_evulation(model4, Testing_Set)

In [ ]:
EvaluationValues4

In [ ]:
Real4, Predictions4 = generate_real_prediction_classes(model4)

In [ ]:
ConfusionMatrix4 = confusion_matrix_(Real4, Predictions4)

In [ ]:
Positive4 = Generate_Positive(ConfusionMatrix4)
TruePositive4 = Positive4.true_positive()
FalsePostive4 = Positive4.false_positive()

In [ ]:
Negative4 = Generate_Negative(ConfusionMatrix4)
TrueNegative4 = Negative1.true_negative()
FalseNegative4 = Negative1.false_negative()

In [ ]:
Performance4 = Performance_Metrics(TruePositive4, TrueNegative4 ,FalsePostive4, FalseNegative4)

Accuracy4 = Performance4.accuracy_score()
Precision4 = Performance4.precision_score()
Sensitivity4 = Performance4.sensitivity_score()
Specificity4 = Performance4.specificity_score()
F1Score4 = Performance4.f1_score()
ErrorRate4 = Performance4.err_rate()

PerformanceValues4 = Performance4.display_performance_metrics_values(Accuracy4, Precision4, Sensitivity4, Specificity4, F1Score4, ErrorRate4)

In [ ]:
PerformanceValues4

In [ ]:
predict_image(model4, Predictin_Image1)

In [ ]:
predict_image(model4, Predictin_Image2)

In [ ]:
predict_image(model4, Predictin_Image3)

In [ ]:
predict_image(model4, Predictin_Image4)

In [ ]:
predict_image(model4, Predictin_Image5)

In [ ]:
predict_image(model4, Predictin_Image6)

In [ ]:
predict_image(model4, Predictin_Image7)

In [ ]:
predict_image(model4, Predictin_Image8)

**5. ResNet50**

In [ ]:
name = 'ResNet50'

In [ ]:
with tf.device('GPU'):
        base_model = tf.keras.applications.resnet50.ResNet50( 
                include_top=False,
                weights='imagenet',
                pooling = 'avg',
                )
        RES = Models(Input_Shape, Epochs)
        model5 = RES.tranfer_learning(base_model)

In [ ]:
HistoryResultDF5= model_history_values(model5, name)

In [ ]:
HistoryResultDF5

In [ ]:
plot_model_history(HistoryResultDF5)

In [ ]:
ModelValues5 = ModelValues(HistoryResultDF5)
Loss5, Accuracy5, ValLoss5, ValAccuracy5, LossAccuracyValuesDf5 = ModelValues5.loss_accuracy_values()
MaxLoss5, MaxAccuracy5, MaxValLoss5, MaxValAccuracy5, MaxLossAccuracyValuesDf5 = ModelValues5.loss_accuracy_valuesـmaximum_values()
MinLoss5, MinAccuracy5, MinValLoss5, MinValAccuracy5, MinLossAccuracyValuesDf5 = ModelValues5.loss_accuracy_valuesـminimum_values()

In [ ]:
LossAccuracyValuesDf5

In [ ]:
MaxLossAccuracyValuesDf5

In [ ]:
MinLossAccuracyValuesDf5

In [ ]:
TestingLoss5, TestingAccuracy5, EvaluationValues5 = model_evulation(model5, Testing_Set)

In [ ]:
EvaluationValues5

In [ ]:
Real5, Predictions5 = generate_real_prediction_classes(model5)

In [ ]:
ConfusionMatrix5 = confusion_matrix_(Real5, Predictions5)

In [ ]:
Positive5 = Generate_Positive(ConfusionMatrix5)
TruePositive5 = Positive5.true_positive()
FalsePostive5 = Positive5.false_positive()

In [ ]:
Negative5 = Generate_Negative(ConfusionMatrix5)
TrueNegative5 = Negative5.true_negative()
FalseNegative5 = Negative5.false_negative()

In [ ]:
Performance5 = Performance_Metrics(TruePositive5, TrueNegative5 ,FalsePostive5, FalseNegative5)

Accuracy5 = Performance5.accuracy_score()
Precision5 = Performance5.precision_score()
Sensitivity5 = Performance5.sensitivity_score()
Specificity5 = Performance5.specificity_score()
F1Score5 = Performance5.f1_score()
ErrorRate5 = Performance5.err_rate()

PerformanceValues5 = Performance5.display_performance_metrics_values(Accuracy5, Precision5, Sensitivity5, Specificity5, F1Score5, ErrorRate5)

In [ ]:
PerformanceValues5

In [ ]:
predict_image(model5, Predictin_Image1)

In [ ]:
predict_image(model5, Predictin_Image2)

In [ ]:
predict_image(model5, Predictin_Image3)

In [ ]:
predict_image(model5, Predictin_Image4)

In [ ]:
predict_image(model5, Predictin_Image5)

In [ ]:
predict_image(model5, Predictin_Image6)

In [ ]:
predict_image(model5, Predictin_Image7)

In [ ]:
predict_image(model5, Predictin_Image8)

# All Result

In [ ]:
Result = {
        'Training_Loss':[Loss0, Loss1, Loss2, Loss3, Loss4, Loss5],
         'Training_Accuracy': [Accuracy0, Accuracy1, Accuracy2, Accuracy3, Accuracy4, Accuracy5],
         'Val_Loss': [ValLoss0, ValLoss1, ValLoss2, ValLoss3, ValLoss4, ValLoss5],
         'Val_Accuracy': [ValAccuracy0, ValAccuracy1, ValAccuracy2, ValAccuracy3, ValAccuracy4, ValAccuracy5],
         'Testing_Loss':[TestingLoss0, TestingLoss1, TestingLoss2, TestingLoss3, TestingLoss4, TestingLoss5],
         'Testing_Accuracy':[TestingAccuracy0, TestingAccuracy1, TestingAccuracy2, TestingAccuracy3, TestingAccuracy4, TestingAccuracy5],
         'Accuracy':[Accuracy0, Accuracy1, Accuracy2, Accuracy3, Accuracy4, Accuracy5],
         'Precision':[Precision0, Precision1, Precision2, Precision3, Precision4, Precision5],
         'Sensitivity':[Sensitivity0, Sensitivity1, Sensitivity2, Sensitivity3, Sensitivity4, Sensitivity5],
         'Specificity':[Specificity0, Specificity1, Specificity2, Specificity3, Specificity4, Specificity5],
         'F1Score': [F1Score0, F1Score1, F1Score2, F1Score3, F1Score4, F1Score5],
         'ErrorRate':[ErrorRate0, ErrorRate1, ErrorRate2, ErrorRate3, ErrorRate4, ErrorRate5,]
         }

In [ ]:
AllResult = pd.DataFrame(Result, index = ['Convolutional Neural Network', 'InceptionV3', 'DenseNet121', 'MobileNet', 'Xception', 'ResNet50'])

In [ ]:
AllResult